<a href="https://colab.research.google.com/github/ZhiyuanQ/Stage/blob/main/Token_Classification%E7%AC%A6%E5%90%88%E8%87%AA%E5%B7%B1%E4%BB%BB%E5%8A%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tout d'abord il faut démarrer un environnement d'exécution hébergé. 

Installation de 🤗 Transformers and 🤗 Datasets. `seqeval` est utile pour le calcul de quelques métriques d'évaluation. Pour lancer l'installation, décommenter les lignes ci-dessous et lancere l'exécution. 

In [1]:
! pip install transformers
! pip install datasets
! pip install seqeval
! pip install huggingface_hub
! pip install torch
! pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 21.9 MB/s 
     |████████████████████████████████| 6.6 MB 34.0 MB/s 
     |████████████████████████████████| 596 kB 46.3 MB/s 
     |████████████████████████████████| 101 kB 10.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 26.7 MB/s 
     |████████████████████████████████| 140 kB 58.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.3 MB/s 
     |████████████████████████████████| 212 kB 61.6 MB/s 
     |████████████████████████████████| 127 kB 61.4 MB/s 
     |████████████████████████████████| 144 kB 59.8 MB/s 
     |████████████████████████████████| 271 kB 78.3 MB/s 
     |█████████

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then uncomment the following cell and input it:

In [10]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Then you need to install Git-LFS and setup Git if you haven't already. Uncomment the following instructions and adapt with your name and email:

In [12]:
 !apt install git-lfs
 !git config --global user.email "qiuzhiyuan@hotmail.com"
 !git config --global user.name "Zhiyuan Qiu"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Make sure your version of Transformers is at least 4.16.0 since some of the functionality we use was introduced in that version:

In [11]:
import transformers

print(transformers.__version__)

4.20.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/blob/main/examples/images/token_classification.png?raw=1)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use Keras to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [19]:
task = "dd_tags"  # 
batch_size = 16

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [6]:
import datasets
from datasets import load_dataset, load_metric

 The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [15]:
data_dir="/content/sample_data/"
datasets = load_dataset('json', data_files=
                                  {'train': data_dir+'train.json', 
                                   'validation':data_dir+'validation.json',
                                   'test': data_dir+'test.json'
                                   }
                        )

Using custom data configuration default-2d983fb575889316


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-2d983fb575889316/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'dd_tags'],
        num_rows: 20
    })
    validation: Dataset({
        features: ['id', 'tokens', 'dd_tags'],
        num_rows: 20
    })
    test: Dataset({
        features: ['id', 'tokens', 'dd_tags'],
        num_rows: 20
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [17]:
datasets["train"][0]

{'dd_tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 'id': "C'est fou#s17",
 'tokens': ['Joseph',
  '(',
  'en',
  'remontant',
  'dans',
  'sa',
  'chère',
  'Minerva',
  ')',
  'jeta',
  'un',
  'coup',
  'd’',
  'oeil',
  'pour',
  'vérifier',
  'si',
  'tout',
  'son',
  'petit',
  'monde',
  'était',
  'en',
  'ordre',
  '.']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [20]:
datasets["train"].features[f"dd_tags"]

Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

In [23]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=20):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [24]:
show_random_elements(datasets["train"])

,id,tokens,dd_tags
0,C'est fou#s33,"[Elle, était, habillée, tout, en, noir, depuis, la, mort, en, 1915, à, Verdun, de, son, unique, fils, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,C'est fou#s36,"[grommela, mon, ancêtre, .]","[O, O, O, O]"
2,C'est fou#s21,"[Il, prit, néanmoins, le, temps, de, vérifier, que, l’, immense, malle, en, cuir, noir, portant, imprimé, en, blanc, LE, BON, MARCHÉ, était, bien, arrimée, sur, le, toit, de, la, voiture, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,C'est fou#s24,"[—, Bien, ,, monsieur, le, baron, ,, approuva, joyeusement, ce, dernier, .]","[B, I, I, I, I, I, I, O, O, O, O, I]"
4,C'est fou#s34,"[—, Les, voilà, ,, monsieur, le, baron, ,, dit, gentiment, Joseph, .]","[B, I, I, I, I, I, I, I, O, O, O, I]"
5,C'est fou#s29,"[Elle, monta, s’, installer, majestueusement, à, côté, de, Grand-père, .]","[O, O, O, O, O, O, O, O, O, O]"
6,C'est fou#s23,"[Bien, qu’, il, y, eût, une, très, bonne, épicerie, dans, le, village, à, le, pied, de, le, château, de, mes, ancêtres, maternels, ,, elle, commandait, la, plupart, de, ses, courses, pour, l’, été, à, le, Bon, Marché, ,, qui, les, lui, livrait, dans, ce, somptueux, coffre, attendu, avec, impatience, par, notre, célèbre, cuisinière, ,, Louise, ,, partie, la, veille, par, le, train, avec, Paul, ,, le, maître, d’, hôtel, (, et, valet, de, chambre, de, Grand-père, ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,C'est fou#s18,"[Oui, .]","[O, O]"
8,C'est fou#s19,"[Grand-père, décachetait, son, courrier, de, la, banque, F.H., Grand-mère, avait, sorti, son, chapelet, noir, de, son, sac, noir, et, commençait, à, prier, le, Seigneur, pour, l’, âme, de, l’, oncle, Christian, «, mort, à, le, combat, », .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
9,C'est fou#s22,"[Car, Grand-mère, avait, une, manie, .]","[O, O, O, O, O, O]"


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
import torch
import sacremoses
# from transformers import FlaubertTokenizer
from transformers import AutoTokenizer, FlaubertTokenizer

modelname = "flaubert/flaubert_base_uncased"
tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)

#tokenizer = AutoTokenizer.from_pretrained(modelname)


The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [ ]:
import transformers

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

AssertionError: ignored

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [ ]:
tokenizer("salut, voici un exemple !")

{'input_ids': [0, 5793, 14, 1088, 26, 273, 82, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [ ]:
tokenizer(
    ["salut", ",", "voici", "une", "phrase", "déjà", "découpée", "en", "tokens", "."],
    is_split_into_words=True,
)

{'input_ids': [0, 5793, 14, 1088, 29, 3945, 172, 35585, 24, 1007, 371, 1371, 16, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [ ]:
example = datasets["train"][4]
print(example["words"])

['"', 'It', 'is', 'one', 'step', 'short', 'of', 'an', 'emergency', 'situation', ',', '"', 'a', 'police', 'spokesman', 'said', 'via', 'telephone', 'from', 'a', 'command', 'post', 'in', 'the', 'bush', '.']


In [ ]:
tokenized_input = tokenizer(example["words"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '"</w>', 'I', 't</w>', 'is</w>', 'one</w>', 'step</w>', 'short</w>', 'of</w>', 'an</w>', 'emer', 'gency</w>', 'situation</w>', ',</w>', '"</w>', 'a</w>', 'police</w>', 'spokes', 'man</w>', 'said</w>', 'via</w>', 'tele', 'phone</w>', 'from</w>', 'a</w>', 'command</w>', 'post</w>', 'in</w>', 'the</w>', 'bush</w>', '.</w>', '</s>']


Here the words "It", "emergency" have been split in two subtokens, as "telephone" or "spokesman".

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [ ]:
len(example[f"dd"]), len(tokenized_input["input_ids"])

(26, 32)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [ ]:
print(tokenized_input.word_ids())

ValueError: ignored

As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [ ]:
# word_ids = tokenized_input.word_ids()
# words_ids contient la liste des index des tokens originaux, pour chaque fragment de tokens obtenu avec le tokeniser

# 1,1 : correspond à I+t
# 8,8 : correspond à emer+gency
# 14,14 : correspond à spokes+man
# 17,17 : correspond à tele+phone

# Comme word_ids() ne fonctionne pas avec ce tokenizer il faudra créer une fonction word_ids() 
# qui renvoie les index des mots pour chaque token obtenu par byte length encoding
# là je l'ai fait à la main
word_ids=[None,0,1,1,2,3,4,5,6,7,8,8,9,10,11,12,13,14,14,15,16,17,17,18,19,20,21,22,23,24,25,None]
aligned_labels = [-100 if i is None else example[f"dd"][i] for i in word_ids]
print(aligned_labels)
print(len(aligned_labels), len(tokenized_input["input_ids"]))

[-100, 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', -100]
32 32


Here we set the labels of all special tokens to -100 (the index that is ignored by the loss function) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. Both strategies are possible with this script; simply change the value of this flag to swap between them.

In [ ]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["words"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"dd"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # ICI TU VAS UTILISER TA FONCTION word_ids()
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [ ]:
tokenize_and_align_labels(datasets["train"][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-6daba6276a6f6057.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-f4b2a208e343ba4a.arrow
Loading cached processed dataset at /home/matt/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6/cache-d25c3236a8db76d3.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

In [ ]:
tokenized_datasets["train"]["labels"][0]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `TFAutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [ ]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list)
)

2022-01-28 16:29:52.803150: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 16:29:52.809743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 16:29:52.810442: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-28 16:29:52.811930: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

After all of the Tensorflow initialization messages, we see a warning that is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To compile a Keras `Model`, we will need to set an optimizer and our loss function. We can use the `create_optimizer` function from Transformers. Here we tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. This function will also create and apply a learning rate decay schedule for us.

In [ ]:
from transformers import create_optimizer

num_train_epochs = 3
num_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

Now we define our loss and compile our model. Note that most models on the Hub compute loss internally - we can just pass nothing to the loss argument if we want to use the internal loss instead.

This is an unusual quirk of TensorFlow models in 🤗 Transformers, so it's worth elaborating on in a little more detail. All 🤗 Transformers models are capable of computing an appropriate loss for their task internally (for example, a CausalLM model will use a cross-entropy loss). To do this, the labels must be provided in the input dict (or equivalently, in the `columns` argument to `to_tf_dataset()`), so that they are visible to the model during the forward pass.

This is quite different from the standard Keras way of handling losses, where labels are passed separately and not visible to the main body of the model, and loss is handled by a function that the user passes to `compile()`, which uses the model outputs and the label to compute a loss value.

The approach we take is that if the user does not pass a loss to `compile()`, the model will assume you want the **internal** loss. If you are doing this, you should make sure that the labels column(s) are included in the **input dict** or in the `columns` argument to `to_tf_dataset`.

If you want to use your own loss, that is of course possible too! If you do this, you should make sure your labels column(s) are passed like normal labels, either as the **second argument** to `model.fit()`, or in the `label_cols` argument to `to_tf_dataset`. 

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


Now we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels. Note that our data collators support multiple frameworks, so ensure you set `return_tensors='tf'` to get `tf.Tensor` outputs - you don't want to forget it and end up with a pile of `torch.Tensor` messing up your Tensorflow code!

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="tf")

Now we need to apply this data collator to our data as it's loaded. The easiest way to do this is with the `to_tf_dataset()` Dataset method. This method converts the dataset into a `tf.data.Dataset` that Keras understands.

In [ ]:
train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
validation_set = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


Now, let's think about metrics. The `seqeval` framework will give us a nice set of metrics like accuracy, precision, recall and F1 score, and all we need to do is pass it some predictions and labels. But if these aren't Keras Metric objects, how can we use them in the middle of our training loop? The answer is the `KerasMetricCallback`. This callback takes a function and computes it on the predictions from the validation set each epoch, printing it and logging the returned value(s) for other callbacks like `TensorBoard` and `EarlyStopping`. 

This allows much more flexibility with the metric computation functions - you can even include Python code that would be impossible to compile into your model, such as using the `tokenizer` (which is backed by Rust!) to decode model outputs to strings and compare them to the labels. We won't be doing that here, but it's essential for metrics like `BLEU` and `ROUGE` used in tasks like translation.

So how do we use `KerasMetricCallback`? It's straightforward - we simply define a function that takes the `predictions` and `labels` from the validation set and computes a dict of one or more metrics, then pass that function and the validation data to the callback. Note that we discard all predictions where the label is `-100` - this indicates a missing label, either because there's no label for that token, or the token is a padding token.

In [ ]:
import numpy as np
from transformers.keras_callbacks import KerasMetricCallback

metric = load_metric("seqeval")
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=validation_set
)

Now we can train our model. We can also add a callback to sync up our model with the Hub - this allows us to resume training from other machines and even test the model's inference quality midway through training! If you don't want to do this, simply remove the callbacks argument in the call to `fit()`.

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-{task}"

tensorboard_callback = TensorBoard(log_dir="./tc_model_save/logs")

push_to_hub_callback = PushToHubCallback(
    output_dir="./tc_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

callbacks = [metric_callback, tensorboard_callback, push_to_hub_callback]

model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

/home/matt/PycharmProjects/notebooks/examples/tc_model_save is already a clone of https://huggingface.co/Rocketknight1/distilbert-base-uncased-finetuned-ner. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/3


/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Using `-1` to mask the loss for the token is deprecated. Please use `-100` instead.
  return py_builtins.overload_of(f)(*args)
/home/matt/miniconda3/envs/tensorflow28/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:707: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  5/877 [..............................] - ETA: 34s - loss: 1.9113  WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0291s vs `on_train_batch_end` time: 0.0394s). Check your callbacks.


877/877 [==============================] - 64s 66ms/step - loss: 0.1933 - val_loss: 0.0697 - precision: 0.9065 - recall: 0.9236 - f1: 0.9149 - accuracy: 0.9803
Epoch 2/3
  5/877 [..............................] - ETA: 30s - loss: 0.0676

...

877/877 [==============================] - 45s 51ms/step - loss: 0.0541 - val_loss: 0.0601 - precision: 0.9210 - recall: 0.9311 - f1: 0.9260 - accuracy: 0.9824
Epoch 3/3
877/877 [==============================] - 44s 50ms/step - loss: 0.0342 - val_loss: 0.0595 - precision: 0.9240 - recall: 0.9328 - f1: 0.9284 - accuracy: 0.9830


Upload file tf_model.h5:   0%|          | 32.0k/253M [00:00<?, ?B/s]

Upload file logs/validation/events.out.tfevents.1643387434.matt-TRX40-AORUS-PRO-WIFI.57182.1.v2: 100%|########…

Upload file logs/train/events.out.tfevents.1643387397.matt-TRX40-AORUS-PRO-WIFI.57182.0.v2:   2%|2         | 3…

To https://huggingface.co/Rocketknight1/distilbert-base-uncased-finetuned-ner
   79fe9e3..8105eab  main -> main

   79fe9e3..8105eab  main -> main



If you used the callback above, you can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained("your-username/my-awesome-model")
```